In [1]:
#NEW for xml file generation:
import datetime
import xml.etree.ElementTree as etree
from xml.etree.ElementTree import ElementTree
from xml.etree.ElementTree import Element
from xml.etree.ElementTree import parse
from xml.etree.ElementTree import tostring
from xml.etree.ElementTree import fromstring

from xml.dom import minidom

class CreateXML:    
    
    def __init__(self, component, ID = '', institution = ''):
        self.position = component.center
        self.degrees = component.deg
        self.heights = component.hgt
        self.notes = component.notes
        self.rating = component.finalRating
        self.ID = ID
        self.institution = institution
        self.debug = False
        self.createDictionary()
        
    def createDictionary(self):
        self.dict = {'OGPMEASUREMENT': 
                    {'ID': self.ID,
                     'INSTITUTION': self.institution,
                     'PLACEMENT_ANGLE': {'FIDUCIAL1': self.degrees[0], 'FIDUCIAL2': self.degrees[1]},
                     'PLACEMENT_POSITION': {'X': self.position[0], 'Y': self.position[1]},
                     'HEIGHTS': {'MEAN': self.heights[0], 'MIN': self.heights[1], 'MAX': self.heights[2]},
                     'GRADE': self.rating,
                     'COMMENT': self.notes}}
        
    def generateXml(self, input_dict):
        # Generate XML ElementTree from input_dictionary, and return the ElementTree.
        # Note:  does not save XML file!
        # Note:  input_dict is a list of all items contained by the ROOT.  ROOT is not included, but 
        #        is generated automatically.

        root = Element('ROOT')
        root.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
        tree = ElementTree(root)
        if self.debug:
            print("CALLING generateXml().  Full dict is:")
            print(input_dict)

        for item_name, item in input_dict.items():
            # Note:  Need to add a case if one of the items is a list.
            # For files where multiple DATA_SETs w/ the same name are needed.
            # ...problem:  Don't necessarily know how many DATA SETS in advance.
            # (Note:  Might not actually want this)
            if self.debug:
                print("  List not found in generateXml.  Calling dict to element on...")
            child = self.dictToElement(item, item_name)
            root.append(child)
        return tree
    
    def dictToElement(self, input_dict, element_name, data_set_index=None):
        # Reads a dictionary, and returns an XML element 'element_name' filled with the contents of the current object
        # structured according to that dictionary.
        # NOTE:  This must be able to work recursively.  I.e. if one of the objs in the input_dict is a dict,
        #    it reads *that* dictionary and creates an element for it, +appends it to current element.  Etc.
        if self.debug:
            print("***dictToElement:*** element {}, input dict is".format(element_name))
            print('type of input_dict:', type(input_dict))
            print("data_set_index:", data_set_index)

        # NEW:  Add special case for multi-item assembly steps.  Will always appear in DATA_SET dict.
        # If in DATA_SET, data_set_index will NOT be None.  If not None, and a list is found, return element i instead of the usual list handling case.
        # ALT (since need to append all 6 DSs in special case:


        parent = Element(element_name)

        for item_name, item in input_dict.items():
            # CHANGE FOR NEW XML SYSTEM:
            # item is a list (comments), dict (another XML layer), or string (var)
            # If string, use getattr()
            if self.debug:
                print("    dictToElement:  name, dict are:", item_name, item)

            if type(item) == dict:
                #print("  **Dict found.  Calling recursive case...")
                # Recursive case: Create an element from the child dictionary.
                # "Remember" whether currently in a DATA_SET
                child = self.dictToElement(item, item_name, data_set_index=data_set_index)
                parent.append(child)
            elif isinstance(item, list):  #type(item) == list:
                # Base case 1:  List of comments.  Create an element for each one.
                if self.debug:
                    print("    Found list: ", type(item))

                # If currently in a DATA_SET, need to treat differently!
                if data_set_index != None:
                    #print("\nFOUND DATA_SET LIST!  Adding item i:", data_set_index, str(getattr(self, item, None)[data_set_index]), '\n')
                    child = Element(item_name)
                    child.text = str(item[data_set_index])
                    parent.append(child)
                elif len(item) == []:
                    # If empty, need to add a placeholder so load() knows to add an empty list
                    child = Element(item_name)
                    child.text = '[]'
                    parent.append(child)
                else:
                    for comment in item:
                        child = Element(item_name)
                        child.text = str(comment)
                        parent.append(child)
            else:
                if self.debug:
                    print("    INSERTING BASE ITEM", item_name, item)
                # Base case 2:
                child = Element(item_name)
                child.text = str(item)  # item should be the var name!
                # Fill attrs that don't exist w/ None
                # This SHOULD work with properties!
                if self.debug:
                    print("    Creating element {} with contents {}".format(item_name, child.text))
                parent.append(child)

        return parent

    # NEWLY REWORKED
    # Should be fully general--all objects can use this if XML_STRUCT_DICT works.
    # Need separate implementation for tools?
    # NOTE:  Can pass new_struct_dict if multiple XML files must be saved for a part/assembly step.
    def save(self, saveDir, name):  #save_xml(self, xml_tree):
        # Get upload XML struct from self.XML_UPLOAD_DICT
        if self.ID == '' or self.institution == '': 
            print('Cannot make XML file without a user or institution!')
            return
        # Generate XML tree:
        struct_dict = self.dict
        xml_tree = self.generateXml(struct_dict)

        # Save xml file:
        # Store in same directory as .json files, w/ same name:
        filedir, filename = saveDir, name  # self.ID)  #This should be unnecessary...
        filename = filename.replace('.xml', '_upload.xml')
        print("Saving UPLOAD XML file to ", filedir+'/'+filename)
        root = xml_tree.getroot()
        xmlstr = minidom.parseString(tostring(root)).toprettyxml(indent = '    ')  #tostring imported from xml.etree.ElementTree
        xmlstr = xmlstr.replace("version=\"1.0\" ", "version=\"1.0\" encoding=\"UTF-8\" standalone=\"yes\"")
        with open(filedir+'/'+filename, 'w') as f:
            f.write(xmlstr)